In [67]:
import sys
import pandas as pd

sys.path.append('../../apple-model-app/')

from helpers import *
import dash_table.FormatTemplate as FormatTemplate
import pandas as pd

# -*- coding: utf-8 -*-
def country_flag(data):
    
    countries = {
    'Canada' : '🇨🇦',
    'USA' : '🇺🇸',
    'Sweden' : '🇸🇪',
    'Finland' : '🇫🇮',
    'Germany' : '🇩🇪',
    'Czech Rep.' : '🇨🇿',
    'Latvia' : '🇱🇻',
    'Russia' : '🇷🇺',
    'Denmark' : '🇩🇰',
    'Switzerland' : '🇨🇭',
    "Norway" : '🇳🇴',
    'Belarus' : '🇧🇾',
    'Italy' : '🇮🇹',
    'U.K.' : '🇬🇧'
    }
    
    try:
        emoji = [countries[d.strip()] for d in data]

        return '/'.join(emoji)
    except:
        return ''


In [68]:
draft_year = 2020

In [ ]:
df = pd.read_csv('data/draft-value.csv')

(draft_styles, _) = rankings_background_color_bins(df[df.base_year == 'DY'],
                                                n_bins=100, columns=[
                                                    'nhl_expected_value', 'xpick_value',
                                                    'nhl_ceiling', 'Projected Rate',
                                                    'surplus', ])

(draft_styles_, _) = rankings_background_color_bins(df[df.base_year == 'DY'], n_bins=100, columns=['real_season_age'], ascending=True)

(_, legend) = legend_background_color_bins(pd.DataFrame([i for i in range(1,101)],
                                                            columns=['percentile']),
                                                columns=['percentile'], n_bins=11)
                                                

In [82]:
def get_draft_table(draft_year):

    index_cols = [
        {'id' : 'draft_round' , 'name' : ["", "Round"]},
        {'id' : 'draft_pick', 'name' : ["", "Pick"]}, 
        {'id' : 'draft_team', 'name' : ["", "Team"]}, 
        {'id' : 'player', 'name' : ["", "Name"]},
        {'id' : 'nation', 'name' : ["", "Nation"]}, 
        {'id' : 'position', 'name' : ["", "Position"]}, 
        {'id' : 'height', 'name' : ["", "Height"]}, 
        {'id' : 'weight', 'name' : ["", "Weight"]}, 
        {'id' : 'real_season_age', 'name' : ["", "Age"]},
    ]

    draft_year_cols = [
        {'id' : 'league' ,'name' : ["Draft Year", "League"]}, 
        {'id' : 'gp' ,'name' : ["Draft Year", "GP"]}, 
        {'id' : 'g' ,'name' : ["Draft Year", "G"]}, 
        {'id' : 'a' ,'name' : ["Draft Year", "A"]}, 
        {'id' : 'tp' ,'name' : ["Draft Year", "PTS"]}, 
        {'id' : 'ppg' ,'name' : ["Draft Year", "PPG"]}, 
        {'id' : 'draft_value_range' ,'name' : ["Draft Year", "Draft Range"]}, 
        {'id' : 'draft_upside' ,'name' : ["Draft Year", "Draft Upside"]},
    ] 

    apple_cols = [
        {'id' : 'nhl_expected_value', 'name' : ["APPLE", "xValue"] },
        {'id' : 'xpick_value', 'name' : ["APPLE", "xValue by Pick"]},
        {'id' : 'nhl_ceiling', 'name' : ["APPLE", "Points"] },
        {'id' : 'Projected Rate', 'name' : ["APPLE", "Projected Rate"] },
        {'id' : 'Projected Role', 'name' : ["APPLE", "Projected Role"]},
        {'id' : 'surplus', 'name' : ["APPLE", "Surplus"] },
        {'id' : 'surplus_relative', 'name' : ["APPLE", "Surplus Rel."]}
    ]

    nhl_cols = [
        {'id' : 'draft_plus_5_gp', 'name' : ["NHL", "GP"]},
        {'id' : 'draft_plus_5_g', 'name' : ["NHL", "G"]},
        {'id' : 'draft_plus_5_a', 'name' : ["NHL", "A"]},
        {'id' : 'draft_plus_5_points', 'name' : ["NHL", "PTS"]},
        {'id' : 'draft_plus_5_ppg', 'name' : ["NHL", "PPG"]},
    ]


    data = df[(df.draft_year == draft_year)
                   & (df.base_year == 'DY')]

    data['nation'] = data.nation.apply(lambda x : country_flag(x.split('/')))

    data.loc[:, 'weight'] = (data.weight * 2.2).astype(int) # covert to lbs
    data.loc[:, 'height'] = data.height.apply(get_imperial_height) # covert to feet and inches
    data['Projected Rate'] = data['Projected Rate'].fillna(0.0)

    table = dash_table.DataTable(
            id='draft-rankings',
            columns=index_cols + draft_year_cols + apple_cols + nhl_cols,
            data=data.sort_values('draft_pick').round(2).to_dict('records'),
            style_data_conditional=draft_styles + draft_styles_,
            style_table={
                    'width': '96%', # both are needed to make virtualization table fill screen
                    'maxWidth' : '98%',
                    'padding-left' : '2%',
                },
            style_cell={'textAlign': 'center',
                        "border": "4px solid white",
                        'whiteSpace' : 'normal',
                        "minWidth": "40px",
                       },
            sort_action="native",
            style_header_conditional=[
                { 'if': { 'header_index': 0 }, 'backgroundColor': 'white' , "border": "4px solid black",},
            ],
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold'
            },
            merge_duplicate_headers=True,
            tooltip={
                    'nhl_ceiling': {'type': 'text', 'value': 'NHL Point Ceiling by Draft + 5'},
                    'nhl_expected_value': {'type': 'text', 'value': 'Expected Cumulative NHL Points up to Draft + 5'}}
        )
    
    return table

In [84]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc

app = JupyterDash(__name__)
app.layout = html.Div([
    dbc.Row(dcc.Markdown(f'''
                        ### *{draft_year} NHL Draft Rankings* 
                            ''', 
                         style = {'font-family': 'monospace', 'font-size' : '24px'}), justify='center'),
    dbc.Row([
        dbc.Col(html.H4('Position Percentile Rank')),
        dbc.Col(legend)]),
    get_draft_table(draft_year)
                       ]
                      )

if __name__ == '__main__':
    app.run_server(mode='inline', port='8787')

/Users/connorjung/.pyenv/versions/apple/lib/python3.7/site-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/connorjung/.pyenv/versions/apple/lib/python3.7/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/Users/connorjung/.pyenv/versions/apple/lib/python3.7/site-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_